Jon Nordby <jonnord@nmbu.no>


## Theory

    insertion sort
    merge sort
    quicksort
    heapsort


TODO: list all the algorithms involved, expectations about runtime.
Best-case, worst-case, average-case.
Other properties. Inplace, stable sort

https://en.wikipedia.org/wiki/Sorting_algorithm


## Methods


The methods section should include

    your algorithm implementations 
    test for correctness
    functions for generating test data
    functions to collect benchmark data

In addition to the actual code, the section should contain brief descriptions of the code. Provide docstrings, but avoid inline comments in Python code.

Also describe you strategy for collecting benchmark data.


### Correctness

Correctness should be checked against Python as oracle

TODO: test correctness. Functions should at least work for Python list and NumPy array. Both numbers and strings


### Test data generation

Test data should be automatically generated. As default items to be sorted, use NumPy arrays filled with integers, but also use strings. Test at least with the following types of data

    sorted in ascending order
    sorted in descending order
    identical values
    randomly shuffled data
    real-life data



### Insertion sort


TODO: implement

### Merge sort

TODO: implement

### Quicksort

TODO: implement

### Heapsort

TODO: implement


### Test data generation




## Results

Results should contain notebook cells generating benchmark data and graphics showing that data, including brief descriptions of the observations.

ou should attempt going to at least 1 million entries, except for insertion sort, where you probably need to stop for much smaller data sizes

Plot execution times for different N, for each algo

Maybe plot distribution of times for a given N and algo. Indicator of measurement variation

TODO: compare against include python stardard `.sort()` / `sorted()` ?
TODO: compare against Numpy sort?
TODO: compare methods using Numba/Cython

## Discussion

Discuss your results in light of the expectations from the theoretical runtime analysis we performed in class for different types of test data. Comment on unexpected or surprising results. Try to interpret your results.

## Conclusions